In [1]:
from sklearn.inspection import permutation_importance
from joblib import load
import pandas as pd

## Vanilla Feature Importance

In [5]:
feature_importance = pd.read_csv("final_RF_feature_importance.csv")

In [ ]:
feature_importance.sort_values(by=['Rank']).head()

## Permutation Importance
Explanation here: https://explained.ai/rf-importance

In [20]:
test = pd.read_parquet("final_test_2.parquet").sample(n = 100000, random_state = 55)

In [16]:
rf = load('RF_final.joblib')

/home/ubuntu/miniconda3/envs/perm/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22 when using version 0.23.dev0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/ubuntu/miniconda3/envs/perm/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.22 when using version 0.23.dev0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [17]:
neighbor_cols = [f'neighbor_{n}' for n in range(25)]
input_cols = ['imputed_epa_pm25_value', 'imputed_hum', 
              'imputed_temperature', 'wind_x','wind_y', 
              'ndvi', 'elevation'] + neighbor_cols

In [21]:
result = permutation_importance(rf, test[input_cols], test['2_5um'], random_state = 1210)

## Putting them together

In [34]:
feature_importance['mean_perm'] = result['importances_mean']
feature_importance['std_perm'] = result['importances_std']

In [50]:
neighbors_total = feature_importance[feature_importance.Feature.map(lambda x: x.startswith("neighbor"))] \
    .sum().set_value('Feature','neighbors_total')

wind_total = feature_importance[feature_importance.Feature.map(lambda x: x.startswith("wind"))] \
    .sum().set_value('Feature','wind_total')

<ipython-input-50-ed485721f2ee>:1: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  neighbors_total = feature_importance[feature_importance.Feature.map(lambda x: x.startswith("neighbor"))] \
<ipython-input-50-ed485721f2ee>:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  wind_total = feature_importance[feature_importance.Feature.map(lambda x: x.startswith("wind"))] \


In [42]:
f = feature_importance.Feature.map(lambda x: x.startswith("neighbor_") or x.startswith("wind_"))

In [69]:
rank_df = feature_importance[~f] \
    .append(pd.DataFrame(neighbors_total).T) \
    .append(pd.DataFrame(wind_total).T) \
    .drop(columns=['Rank']) \
    .assign(perm_importance = lambda df_: df_.mean_perm / df_.mean_perm.sum(),
            feature_importance_rank = lambda df_: df_.Importance.rank(ascending = False).astype(int),
            permutation_importance_rank = lambda df_: df_.mean_perm.rank(ascending = False).astype(int)) \
    .rename(columns={'Importance':'feature_importance', 'mean_perm':'permutation_importance'}) \
    .sort_values(by=['permutation_importance_rank','feature_importance_rank']) \
    .reset_index(drop = True)

In [71]:
rank_df[["Feature","feature_importance", "permutation_importance","feature_importance_rank", "permutation_importance_rank"]]

,Feature,feature_importance,permutation_importance,feature_importance_rank,permutation_importance_rank
0,imputed_epa_pm25_value,0.557002,1.02331,1,1
1,neighbors_total,0.160226,0.143997,2,2
2,imputed_temperature,0.0490909,0.102274,7,3
3,ndvi,0.0524636,0.096655,6,4
4,imputed_hum,0.0539146,0.096074,5,5
5,elevation,0.053955,0.0860544,4,6
6,wind_total,0.0733475,0.0644375,3,7
